In [14]:
import pandas as pd
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from PIL import Image

In [15]:
def get_sample(index, frame):
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data

In [16]:
class ImageDataset_2(Dataset):
    """
    Класс с торча, возвращает датасет из картинок и таргетов
    Принимает на вход директорию и датафрейм из предыдущих пунктов
    """
    def __init__(self, image_dir, data, transform=None):
        self.image_dir = image_dir
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx, 0])  
        image = Image.open(f'{img_name}.jpg').convert("RGB")
        if self.transform:
            image = self.transform(image)
        target = torch.tensor(self.data.iloc[idx, 1:].values.astype('float32'), dtype=torch.float32)
        file=self.data.iloc[idx, 0]
        #target = torch.tensor(self.data.iloc[idx, 1:].values.astype('int'), dtype=torch.float32)
        return image, target, file

In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [18]:
def prepare_data_2(data):
    """
    Подготовка датафрейма с таргетами
    Возвращает подготовленный датафрейм
    """
    # B=find_empty_indexes_2(data)#Поиск id с отсутствующими файлами для исключения
    # print(B)
    # df=data[~data['file_name'].isin(B)]#Фильтрация данных
    #Удаление всех столбцов, кроме id и таргетов
    df=data.drop(['Влияние ствола скважины_details', 'Радиальный режим_details', 'Линейный режим_details','Билинейный режим_details','Сферический режим_details','Граница постоянного давления_details','Граница непроницаемый разлом_details'],axis=1)
    return df

In [19]:
class PositionalEncoding(nn.Module):
    """
    Классическое позиционное кодирование (NLP-стиль).
    """
    def __init__(self, d_model, max_len=4000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() 
                             * -(torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe / (d_model ** 0.5)
        pe = pe.unsqueeze(0)    # [1, max_len, d_model]
        self.register_buffer('pe', pe) 

    def forward(self, x):
        """
        x shape: [B, T, d_model]
        Добавляем позиционное кодирование к x.
        """
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        # print(f"Encoded x: {x}")
        return x

class TransformerFlowModel(nn.Module):
    """
    Пример трансформера для задач:
      - 8 бинарных признаков (классификация)
      - 7 регрессионных выходов
    С учётом attention mask (padding).
    """
    def __init__(self,
                 input_dim=3,     # (delta_p, p_, log_time) например
                 d_model=64,
                 nhead=4,
                 num_layers=2,
                 dim_feedforward=128,
                 dropout=0.1,
                 n_class=8,
                 n_reg=5):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        # self.pos_encoder = PositionalEncoding(d_model)
        torch.nn.init.xavier_uniform_(self.input_proj.weight, gain=1.0)
        torch.nn.init.zeros_(self.input_proj.bias)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
                                                   nhead=nhead,
                                                   dim_feedforward=dim_feedforward,
                                                   dropout=dropout,
                                                   activation="relu",
                                                   batch_first=True)  
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, num_layers=num_layers)

        self.pool = nn.AdaptiveAvgPool1d(1)  # mean-pool по временной оси

        self.class_head = nn.Linear(d_model, n_class)
        self.reg_head   = nn.Linear(d_model, n_reg)

    def forward(self, x, src_key_padding_mask=None):
        """
        x: [B, T, input_dim]
        src_key_padding_mask: [B, T], True = игнорируем (паддинг)
        """
        # 1) Линейная проекция входа -> d_model
        x_proj = self.input_proj(x)  # [B, T, d_model]
        if torch.isnan(x_proj).any():
            print("NaN after input projection")
        # 2) Позиционное кодирование
        # x_encoded = self.pos_encoder(x_proj)  # [B, T, d_model]
        # if torch.isnan(x_encoded).any():
            # print("NaN after positional encoding")
        # 3) Пропускаем через энкодер
        #    Важно: указываем mask=... (или src_key_padding_mask=...).
        #    Маска должна быть типа bool, shape [B, T]
        x_trans = self.transformer_encoder(
            # x_encoded,
            x_proj,
            src_key_padding_mask=src_key_padding_mask
        )  # [B, T, d_model]
        if torch.isnan(x_trans).any():
            print("NaN after transformer")

        # 4) Pooling, [B, d_model]
        x_trans_perm = x_trans.permute(0, 2, 1)  # -> [B, d_model, T]
        pooled = self.pool(x_trans_perm).squeeze(-1)  # [B, d_model]
        if torch.isnan(pooled).any():
            print("NaN after pooling")

        # 5) Выход
        class_logits = self.class_head(pooled)  # [B, 8]
        reg_output   = self.reg_head(pooled)    # [B, 7]
        if torch.isnan(pooled).any():
            print("NaN after class_logits")
        return class_logits, reg_output


class ConvTransformer(nn.Module):
    def __init__(self, d_model=32, nhead=2, num_layers=1, dim_feedforward = 128, in_channels = 2):
        super().__init__()
        # 1) Conv1d: in_channels=2 (delta_p, p_), out_channels=d_model
        #    kernel_size=3 -> можно менять
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=d_model, kernel_size=31, padding=15, groups=in_channels)
        self.in_channels = in_channels
        # 2) TransformerEncoder (упрощённый)    
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=0.0,
            activation='relu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # 3) Heads для классификации (8) и регрессии (7)
        # self.class_head = nn.Linear(d_model, 8)
        self.reg_preload   = nn.Linear(d_model, 64)
        self.reg_activate = nn.LeakyReLU(0.3)
        self.reg_head   = nn.Linear(64, 1)

    def forward(self, x, src_key_padding_mask=None):
        """
        x: [B, T, 2]
        src_key_padding_mask: [B, T] (bool), True=игнорировать позицию
        """
        # -- (A) Свёртка --
        # Conv1d ожидает [B, C, T], значит permute:
        x = x.permute(0, 2, 1)  # -> [B, 2, T]

        # Прогон через conv:
        # Выход будет [B, d_model, T]
        x_conv = self.conv(x)  # [B, d_model, T]

        # -- (B) Для Transformer делаем [B, T, d_model]
        x_conv = x_conv.permute(0, 2, 1)  # [B, T, d_model]

        # -- (C) Прогон через TransformerEncoder, с учётом mask
        x_trans = self.transformer_encoder(
            x_conv,
            src_key_padding_mask=src_key_padding_mask
        )  # [B, T, d_model]

        # -- (D) Возьмём, например, средний вектор по времени
        x_pooled = x_trans.mean(dim=1)  # [B, d_model]

        # -- (E) Предсказываем 8 бинарных признаков + 7 регрессий
        # class_logits = self.class_head(x_pooled)  # [B, 8]
        reg_out      = self.reg_head(self.reg_activate(self.reg_preload(x_pooled)))    # [B, 7]
        return reg_out
        # return class_logits, reg_out



class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe:pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        x = get_sample(idx, self.siam_dataset_describe)
        # x = torch.from_numpy(get_sample(idx, self.siam_dataset_describe).to_numpy(dtype=np.float32)) #.to_numpy(dtype=np.float64)
        # t = x["time"].to_numpy(dtype=np.float64)
        def calculate_log_pressure(pressure):
            """Calculate log of pressure for log-log plots"""
            # Ensure all pressure values are positive
            min_positive = np.min(pressure[pressure > 0]) / 10 if any(pressure > 0) else 1e-10
            adjusted_pressure = np.maximum(pressure, min_positive)
            return np.log10(adjusted_pressure)
        
        x["time"] = calculate_log_pressure(x["time"])
        x["delta_p"] = calculate_log_pressure(x["delta_p"])
        x["p_"] = calculate_log_pressure(x["p_"]) 
                         # "time",   "delta_p",
        x =  torch.from_numpy(x[["time", "delta_p", "p_"]].to_numpy(dtype=np.float32))
        
        # x =  torch.from_numpy(x[["p_"]].to_numpy(dtype=np.float32))
        if torch.isnan(x).any():
            # print(get_sample(idx, self.siam_dataset_describe))
            mask = ~torch.isnan(x).any(dim=1)
            x = x[~mask]

            
            # print(x)
            # raise Exception("ахтунг в данных")
        if x.shape[0] == 0:
            x = torch.tensor([0,0,0])


        # 7) Возвращаем (X, Y)                                                                                                                                              # не работает если ставить 1 число
        return x, torch.from_numpy(self.siam_dataset_describe.iloc[idx][self.siam_dataset_describe.columns[3:11]].to_numpy(dtype=np.float32)), torch.from_numpy(self.siam_dataset_describe.iloc[idx][self.siam_dataset_describe.columns[11:]].to_numpy(dtype=np.float32))



def collate_fn_with_padding(batch):
    """
    batch: список из (X, y_class, y_reg), где X shape [T_i, input_dim].
    Нужно вернуть:
      padded_X: [B, max_len, input_dim]
      src_key_padding_mask: [B, max_len] (bool)
      y_class: [B, 8]
      y_reg:   [B, 7]
    """
    # 1) Определяем batch_size
    batch_size = len(batch)
    # 2) Находим максимальную длину среди X
    lengths = [sample[0].shape[0] for sample in batch]  # T_i для каждого
    max_len = max(lengths)
    input_dim = batch[0][0].shape[1]

    # 3) Создаём тензоры под результирующие данные
    padded_X = torch.zeros((batch_size, max_len, input_dim), dtype=torch.float)
    # Маска: True = игнорируем => паддинг
    # Изначально False (значит реальная точка), затем выставим True там, где нет реальных данных
    src_key_padding_mask = torch.zeros((batch_size, max_len), dtype=torch.bool)

    y_class_list = []
    y_reg_list = []

    # 4) Копируем данные в паддинг-тензоры
    for i, (X, y_class, y_reg) in enumerate(batch):
        length = X.shape[0]
        padded_X[i, :length, :] = X
        # Для элементов после length делаем mask = True
        if length < max_len:
            src_key_padding_mask[i, length:] = True

        y_class_list.append(y_class)
        y_reg_list.append(y_reg)

    # 5) Склеиваем метки
    y_class_tensor = torch.stack(y_class_list, dim=0)  # [B, 8]
    y_reg_tensor   = torch.stack(y_reg_list, dim=0)   # [B, 7]

    return padded_X, src_key_padding_mask, y_class_tensor, y_reg_tensor

In [20]:
device = 'mps'

In [ ]:
model_class=models.resnet50(pretrained=False)
model_class.fc = nn.Linear(model_class.fc.in_features, 8, bias=True)
model_class.load_state_dict(torch.load('saved_models/finetuned_resnet50_new.pth', weights_only=True, map_location=torch.device(device)))

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [22]:
from copy import deepcopy
regression_layer = []
for i in range(7):
    model = ConvTransformer(d_model=60, nhead=10, num_layers=2, dim_feedforward=128, in_channels=3)
    model.load_state_dict(torch.load(f"saved_models/BESTTransmodelv2_{i}class_all_params_ALLDATA0.pt", weights_only=False, map_location=torch.device(device)))
    model.eval()
    regression_layer.append(deepcopy(model))

# Загрузка данных

In [23]:
frame = pd.read_csv("submit1.csv")
frame = frame[~(frame['file_name']=='846291a2-6475-47ac-8f10-08eac6b93fcb')]
submit_class=prepare_data_2(frame)

In [24]:
testset_equal=ImageDataset_2("imgs_equal_test/test", submit_class, transform=transform)
testloader = DataLoader(testset_equal, batch_size=499, shuffle=False)#Получение итерируемого объекта с тренировочным датасетом

In [25]:
val_data = SiamDataset(frame)
loader = DataLoader(val_data, 
                        batch_size=len(val_data), 
                        shuffle=False, 
                        collate_fn=collate_fn_with_padding)

# Предсказания

In [ ]:
model_class.eval()
with torch.no_grad():
    for images, labels, files in testloader:
        outputs = model_class(images)
        file=np.array(files)

In [27]:
outputs=torch.sigmoid(outputs)
predicted = (outputs > 0.7).int()
p_classes=predicted.cpu().detach().numpy()

In [ ]:
reg_result = []
padded_X, src_mask, _, _ = [d for d in loader][0]
result_array = regression_layer[0](padded_X, src_key_padding_mask=src_mask).cpu().detach().numpy()
for i in range(1, 7):
    result_array = np.concatenate([result_array,  regression_layer[i](padded_X, src_key_padding_mask=src_mask).cpu().detach().numpy()], axis=1)

In [71]:
final_reg = p_classes[:, 1:len(p_classes)] * result_array

In [72]:
final_reg

array([[ 0.76775396,  1.41333961, -0.07928722, ..., -0.        ,
        -0.        , -0.        ],
       [ 0.        , -2.00766897, -0.        , ..., -1.99857104,
        -0.        , -2.00171804],
       [ 0.79261231,  0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       ...,
       [ 0.46920478,  0.89288944, -0.        , ..., -0.        ,
        -2.00378203, -0.        ],
       [ 1.0317564 ,  0.33605567, -0.        , ..., -0.        ,
        -0.        , -1.99082005],
       [ 1.0551399 , -0.        , -0.        , ..., -0.        ,
        -2.0032413 , -0.        ]])

In [73]:
df_classification=pd.DataFrame(p_classes)
df_f=pd.DataFrame(file)
df_reg=pd.DataFrame(final_reg)

df_itog=pd.concat([df_f, df_classification, df_reg], axis=1)
df_itog.columns=frame.columns

In [74]:
df_itog

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,1,1,1,1,0,0,0,0.767754,1.413340,-0.079287,-1.994715,-0.000000,-0.000000,-0.000000
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,1,0,0,1,0,1,0.000000,-2.007669,-0.000000,-0.000000,-1.998571,-0.000000,-2.001718
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,1,0,0,0,0,0,0,0.792612,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,1,0,0,0,0,1,0,1.941281,-0.000000,-0.000000,-0.000000,-0.000000,-2.003519,-0.000000
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,1,1,0,0,1,0,0,1.814303,0.197002,-0.000000,-0.000000,-1.975934,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,6fe55825-442f-4a73-a659-50346c8096d2,0,1,1,0,0,1,1,0,1.778392,-0.099800,-0.000000,-0.000000,-1.983782,-2.003338,-0.000000
495,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,1,1,0,0,1,0,0,2.066915,1.200407,-0.000000,-0.000000,-1.991804,-0.000000,-0.000000
496,903787df-01b6-4308-b037-11c7ba81eb4e,0,1,1,0,0,0,1,0,0.469205,0.892889,-0.000000,-0.000000,-0.000000,-2.003782,-0.000000
497,3c264f8a-df09-4beb-b64f-879c515e88b0,0,1,1,0,0,0,0,1,1.031756,0.336056,-0.000000,-0.000000,-0.000000,-0.000000,-1.990820


In [75]:
df_itog_with_bad = deepcopy(df_itog)

df_itog_with_bad.loc[len(df_itog_with_bad)] = ["846291a2-6475-47ac-8f10-08eac6b93fcb", 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [76]:
df_itog_with_bad

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,1,1,1,1,0,0,0,0.767754,1.413340,-0.079287,-1.994715,-0.000000,-0.000000,-0.000000
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,1,0,0,1,0,1,0.000000,-2.007669,-0.000000,-0.000000,-1.998571,-0.000000,-2.001718
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,1,0,0,0,0,0,0,0.792612,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,1,0,0,0,0,1,0,1.941281,-0.000000,-0.000000,-0.000000,-0.000000,-2.003519,-0.000000
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,1,1,0,0,1,0,0,1.814303,0.197002,-0.000000,-0.000000,-1.975934,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,1,1,0,0,1,0,0,2.066915,1.200407,-0.000000,-0.000000,-1.991804,-0.000000,-0.000000
496,903787df-01b6-4308-b037-11c7ba81eb4e,0,1,1,0,0,0,1,0,0.469205,0.892889,-0.000000,-0.000000,-0.000000,-2.003782,-0.000000
497,3c264f8a-df09-4beb-b64f-879c515e88b0,0,1,1,0,0,0,0,1,1.031756,0.336056,-0.000000,-0.000000,-0.000000,-0.000000,-1.990820
498,2f1769a9-f3d3-4ecb-997a-874db3846b37,0,1,0,0,0,0,1,0,1.055140,-0.000000,-0.000000,-0.000000,-0.000000,-2.003241,-0.000000


In [77]:
df_itog_with_bad.to_csv('submit_test1.csv', index=False)